<a href="https://colab.research.google.com/github/dinaglamshowroom/projet-data_oc/blob/main/Projet4_nettoyage_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Partie I:

Nettoyage des données

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import math as mth
import statsmodels.api as sm
from datetime import datetime
from IPython.display import display
from datetime import date



In [57]:
def save_fig(name):
    path_drive = f"/content/drive/MyDrive/Projet4/graphiques/{name}.png"
    path_git = f"/content/mon_repo_github/figures/{name}.png"
    plt.savefig(path_drive, dpi=300, bbox_inches='tight')
    !cp "{path_drive}" "{path_git}"
    print(f"Graphique enregistré :\n📁 Drive → {path_drive}\n🐙 GitHub → {path_git}")

In [58]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [59]:
import os, glob

DATA_DIR = "/content/drive/MyDrive/Projet_4/Data_P4"
csv_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))

print("Fichiers CSV trouvés :")
for f in csv_files:
    print(f)

Fichiers CSV trouvés :
/content/drive/MyDrive/Projet_4/Data_P4/transactions.csv
/content/drive/MyDrive/Projet_4/Data_P4/products.csv
/content/drive/MyDrive/Projet_4/Data_P4/customers.csv


In [60]:


# 4. Charger tous les CSV dans un dictionnaire
import pandas as pd
dfs = {f.split("/")[-1]: pd.read_csv(f) for f in csv_files}

# 5. Afficher un résumé de chaque CSV (nom + dimensions)
for name, df in dfs.items():
    print(f"{name}: {df.shape}")



transactions.csv: (337016, 4)
products.csv: (3287, 3)
customers.csv: (8623, 3)


Je vérifie mes dataframes

In [61]:
df_customers = dfs["customers.csv"]
df_products = dfs["products.csv"]
df_transactions = dfs["transactions.csv"]

#Visualisation rapide de nos 3 dataframes
print(df_products.info())
print("----")
print(df_customers.info())
print("----")
print(df_transactions.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB
None
----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB
None
----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     337016 non-null  object
 1   date        337016 non-null  o

je Vérifie les valeurs manquantes


*   print(df.isnull().any())   # Quelles colonnes ont des NaN ?
*   print(df.isna().sum())     # Combien de NaN dans chaque colonne ?




In [62]:
print("=== Customers ===")
print(df_customers.isna().any())
print(df_customers.isna().sum())
print("\n=== Products ===")
print(df_products.isna().any())
print(df_products.isna().sum())
print("\n=== Transactions ===")
print(df_transactions.isna().any())
print(df_transactions.isna().sum())


=== Customers ===
client_id    False
sex          False
birth        False
dtype: bool
client_id    0
sex          0
birth        0
dtype: int64

=== Products ===
id_prod    False
price      False
categ      False
dtype: bool
id_prod    0
price      0
categ      0
dtype: int64

=== Transactions ===
id_prod       False
date          False
session_id    False
client_id     False
dtype: bool
id_prod       0
date          0
session_id    0
client_id     0
dtype: int64


je remarque qu'il n'y a aucune valeur manquante dans les 3 dataframes


Nous allons maintenant vérifier la cohérance des données en détéctant les valeurs aberrantes

Commençons par la dataframe df_customers

In [63]:
print("=== Customers ===")
print(df_customers.info())
print("\nDimensions :", df_customers.shape)
print("\nPremières lignes :")
display(df_customers.head())

=== Customers ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB
None

Dimensions : (8623, 3)

Premières lignes :


,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


On voit que les colonnes "client_id" et "sex" sont bien des "object" et que les colonnes "age" et birth" sont bien des nombres.

De plus on a le meme nombres d'entrée dans toutes les colonnes et on a la confirmation qu'il n'a pas de données nulles

In [64]:
df_customers.head()              # allure de la dataframe



,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


On analyse la colonne " clien_id"

In [65]:
df["client_id"].describe()   # stats sur UNE colonne

,client_id
count,8623
unique,8623
top,c_84
freq,1


on analyse la df_customers en regardant les extremes de la colonne "id_client"

In [66]:
#Tri rapide sur les 5 premières lignes du dataframe df_customers (méthode .sort_values())
df_customers.sort_values('client_id', ascending = False).head()


,client_id,sex,birth
8494,ct_1,m,2001
2735,ct_0,f,2001
7358,c_999,m,1964
2145,c_998,m,2001
94,c_997,f,1994


In [67]:
#Tri rapide sur les 5 premières lignes du dataframe df_customers (méthode .sort_values())
df_customers.sort_values('client_id', ascending = True).head()

,client_id,sex,birth
4299,c_1,m,1955
6894,c_10,m,1956
2137,c_100,m,1992
8472,c_1000,f,1966
3426,c_1001,m,1982


on remarque deux **"ct_"** lignes ayant le même age avec un id qui semble different du reste de la colone. Nous allons garder cette information de côté et l'explorer dans la suite.

on analyse la colonne "sex":

In [68]:
# Nombre de valeurs uniques
print("Valeurs uniques de sex :", df_customers["sex"].unique())

# Répartition par sexe
print("\nRépartition par sexe :")
print(df_customers["sex"].value_counts())


Valeurs uniques de sex : ['f' 'm']

Répartition par sexe :
sex
f    4491
m    4132
Name: count, dtype: int64


On remarque qu'il n'y a que deux sexes et la répartition ne semble pas problematique

On analyse maintenant la colonne "age"

In [69]:
import datetime

# Calcul de l'âge
current_year = datetime.datetime.now().year
df_customers["age"] = current_year - df_customers["birth"]

# Âge min et max
print("Âge minimum :", df_customers["age"].min())
print("Âge maximum :", df_customers["age"].max())

# Statistiques générales
print("\nRésumé statistique sur les âges :")
print(df_customers["age"].describe())


Âge minimum : 21
Âge maximum : 96

Résumé statistique sur les âges :
count    8623.000000
mean       46.719123
std        16.919535
min        21.000000
25%        33.000000
50%        46.000000
75%        59.000000
max        96.000000
Name: age, dtype: float64


On remarque qu'il n'y a pas de mineur et que le client le plus agé a 96 ans, ce qui est tout a fait plausible

Passons à la dataframe df_transactions

In [70]:
print("\n=== Transactions ===")
print(df_transactions.info())
print("\nDimensions :", df_transactions.shape)


=== Transactions ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     337016 non-null  object
 1   date        337016 non-null  object
 2   session_id  337016 non-null  object
 3   client_id   337016 non-null  object
dtypes: object(4)
memory usage: 10.3+ MB
None

Dimensions : (337016, 4)


on va revenir sur notre interogation concernant les "ct_" de la colonne "client_id"

on va ensuite mettre la date au bon format car elle est actuellement en "object" au lieu de "datetime"

In [71]:
#Tri rapide sur les 5 premières lignes du dataframe df_transactions (méthode .sort_values())
df_transactions.sort_values('client_id', ascending = False).head()

,id_prod,date,session_id,client_id
298399,T_0,test_2021-03-01 02:30:02.237423,s_0,ct_1
57261,T_0,test_2021-03-01 02:30:02.237439,s_0,ct_1
77758,T_0,test_2021-03-01 02:30:02.237429,s_0,ct_1
2895,T_0,test_2021-03-01 02:30:02.237414,s_0,ct_1
250656,T_0,test_2021-03-01 02:30:02.237431,s_0,ct_1


In [72]:
#Tri rapide sur les 5 dernières lignes du dataframe df_transactions
df_transactions.sort_values('client_id', ascending = False).tail()

,id_prod,date,session_id,client_id
83342,0_1429,2021-10-15 11:28:24.523566,s_105105,c_1
321196,0_1547,2021-09-08 08:27:49.586711,s_86739,c_1
298829,0_1090,2021-12-19 02:44:12.827475,s_136532,c_1
28406,1_713,2021-11-15 20:40:00.586010,s_120172,c_1
126857,0_1571,2022-02-01 01:47:04.355850,s_158128,c_1


In [73]:
df_products.sort_values('price', ascending = False).tail()

,id_prod,price,categ
1211,0_1844,0.77,0
370,0_120,0.66,0
2272,0_528,0.62,0
2355,0_202,0.62,0
731,T_0,-1.00,0


on a la confirmation que le client id "ct_" est bien un test, nous allons donc supprimer ses entrées

In [74]:
df_transactions[df_transactions["client_id"] == "ct_0"]



,id_prod,date,session_id,client_id
5955,T_0,test_2021-03-01 02:30:02.237441,s_0,ct_0
13745,T_0,test_2021-03-01 02:30:02.237443,s_0,ct_0
20470,T_0,test_2021-03-01 02:30:02.237442,s_0,ct_0
26407,T_0,test_2021-03-01 02:30:02.237426,s_0,ct_0
27161,T_0,test_2021-03-01 02:30:02.237434,s_0,ct_0
...,...,...,...,...
328925,T_0,test_2021-03-01 02:30:02.237437,s_0,ct_0
329634,T_0,test_2021-03-01 02:30:02.237438,s_0,ct_0
330454,T_0,test_2021-03-01 02:30:02.237436,s_0,ct_0
332594,T_0,test_2021-03-01 02:30:02.237445,s_0,ct_0


In [75]:
#Suppression des deux lignes client_id ct_1 et ct_0
df_transactions = df_transactions[(df_transactions.client_id != 'ct_0') & (df_transactions.client_id != 'ct_1')]

In [76]:
df_transactions.sort_values('client_id', ascending = False).head()

,id_prod,date,session_id,client_id
113798,0_1467,2021-12-17 03:28:47.129085,s_135545,c_999
77299,1_395,2021-08-23 16:44:27.072834,s_79695,c_999
328970,0_1426,2021-09-10 02:44:13.556163,s_87567,c_999
144925,1_381,2021-08-14 07:48:55.723362,s_75530,c_999
153027,1_251,2021-11-09 09:06:38.818975,s_116988,c_999


Idem pour la dataframe df_customers

In [77]:
df_customers = df_customers[(df_customers.client_id != 'ct_0') & (df_customers.client_id != 'ct_1')]

In [78]:
df_customers.sort_values('client_id', ascending = False).head()

,client_id,sex,birth,age
7358,c_999,m,1964,61
2145,c_998,m,2001,24
94,c_997,f,1994,31
2788,c_996,f,1970,55
7004,c_995,m,1955,70


In [79]:
df_products = df_products[(df_products.id_prod != 'ct_0') & (df_products.id_prod != 'ct_1')]

on va maintenant mettre la date au format datetime

In [80]:
#Conversion des valeurs de la colonne 'date' dans le bon format date (méthode .to_datetime()),
df_transactions["date"] = pd.to_datetime(
    df_transactions["date"],
    format="%Y-%m-%d %H:%M:%S.%f",
    errors="coerce"
)


/tmp/ipython-input-1130472640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transactions["date"] = pd.to_datetime(


In [81]:
print(df_transactions["date"].dtype)


datetime64[ns]


In [82]:
print("\nPériode couverte par les transactions :")
print(df_transactions["date"].min(), " -> ", df_transactions["date"].max())


Période couverte par les transactions :
2021-03-01 00:01:07.843138  ->  2022-02-28 23:59:58.040472


In [83]:
print("\n=== Products ===")
print(df_products.info())
print("\nDimensions :", df_transactions.shape)


=== Products ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB
None

Dimensions : (336816, 4)


on verifie les doublons

In [84]:
print("\nclient_id unique dans customers ?", df_customers["client_id"].is_unique)
print("\nid_prod est unique ?", df_products["id_prod"].is_unique)

# --- Doublons réels dans transactions ---
print("\nDoublons exacts de lignes transactions :", df_transactions.duplicated().sum())

# --- Doublons réels dans customers ---
print("\nDoublons exacts de lignes customers :", df_customers.duplicated().sum())

# --- Doublons réels dans products ---
print("\nDoublons exacts de lignes products :", df_products.duplicated().sum())


client_id unique dans customers ? True

id_prod est unique ? True

Doublons exacts de lignes transactions : 0

Doublons exacts de lignes customers : 0

Doublons exacts de lignes products : 0


In [85]:
# Clients référencés dans transactions mais absents de customers (problème d'intégrité)
invalid_clients = set(df_transactions["client_id"]) - set(df_customers["client_id"])
print("Clients dans transactions mais pas dans customers :", len(invalid_clients))

# Produits référencés dans transactions mais absents de customers (problème d'intégrité)
invalid_products = set(df_transactions["id_prod"]) - set(df_products["id_prod"])
print("Produits dans transactions mais pas dans products :", len(invalid_products))

# Vérifier que des lignes entieres ne soient dupliquées
print("Doublons exacts de lignes transactions :", df_transactions.duplicated().sum())



Clients dans transactions mais pas dans customers : 0
Produits dans transactions mais pas dans products : 1
Doublons exacts de lignes transactions : 0


In [86]:
# Re-normaliser les clés pour éviter les faux positifs (espaces, types)
df_products["id_prod"] = df_products["id_prod"].astype(str).str.strip()
df_transactions["id_prod"] = df_transactions["id_prod"].astype(str).str.strip()

# Recalculer l’ensemble des produits invalides
invalid_products = set(df_transactions["id_prod"]) - set(df_products["id_prod"])
print("Produits invalides (liste) :", invalid_products)

# Voir combien de transactions sont concernées et un aperçu
bad_tx = df_transactions[df_transactions["id_prod"].isin(invalid_products)]
print("Nombre de transactions impactées :", len(bad_tx))
bad_tx.head(10)


Produits invalides (liste) : {'0_2245'}
Nombre de transactions impactées : 103


/tmp/ipython-input-421144351.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transactions["id_prod"] = df_transactions["id_prod"].astype(str).str.strip()


,id_prod,date,session_id,client_id
6235,0_2245,2021-06-17 03:03:12.668129,s_49705,c_1533
10802,0_2245,2021-06-16 05:53:01.627491,s_49323,c_7954
14051,0_2245,2021-11-24 17:35:59.911427,s_124474,c_5120
17486,0_2245,2022-02-28 18:08:49.875709,s_172304,c_4964
21078,0_2245,2021-03-01 00:09:29.301897,s_3,c_580
21761,0_2245,2022-02-11 09:05:43.952857,s_163405,c_1098
22680,0_2245,2021-10-19 00:28:01.920054,s_106841,c_3953
24584,0_2245,2022-02-25 00:08:08.736068,s_170426,c_6236
30886,0_2245,2021-08-22 08:51:27.564509,s_79102,c_6752
31342,0_2245,2021-05-12 03:36:34.586221,s_33316,c_6205


In [87]:
# Assurer des chaînes propres (déjà fait, mais proprement pour éviter l’avertissement)
df_transactions.loc[:, "id_prod"] = df_transactions["id_prod"].astype(str).str.strip()
df_products.loc[:, "id_prod"] = df_products["id_prod"].astype(str).str.strip()

# Chercher des ID proches de "0_2245" dans le référentiel produits
import difflib
candidates = difflib.get_close_matches("0_2245", df_products["id_prod"].unique(), n=5, cutoff=0.8)
print("Candidats proches :", candidates)

# En complément, heuristiques simples
print("IDs commençant par '0_224' :", df_products[df_products["id_prod"].str.startswith("0_224")]["id_prod"].head(20).tolist())
print("IDs se terminant par '2245' :", df_products[df_products["id_prod"].str.endswith("2245")]["id_prod"].head(20).tolist())


Candidats proches : ['0_245', '0_225', '0_224', '0_2295', '0_2294']
IDs commençant par '0_224' : ['0_2249', '0_2246', '0_2247', '0_224', '0_2242', '0_2244', '0_2241', '0_2240', '0_2243', '0_2248']
IDs se terminant par '2245' : []


In [88]:
key_cols = ["session_id", "client_id", "id_prod", "date"]
print("Doublons sur clé composite dans transactions :",
      df_transactions.duplicated(subset=key_cols).sum())

Doublons sur clé composite dans transactions : 0


Constat : un ID produit manquant (0_2245), 103 transactions impactées.

Analyse : correspond probablement à un produit de la série 0_224X, mais non enregistré dans la table products.

Vu le volume des transaction, on decide de supprimer les 103 transactions

In [89]:
df_transactions = df_transactions[df_transactions["id_prod"] != "0_2245"]


In [90]:
invalid_products_post = set(df_transactions["id_prod"]) - set(df_products["id_prod"])
print(invalid_products_post)


set()


Cela confirme que ton intégrité produits ↔ transactions est rétablie

on enregistre tout dans un le dossier google drive pour que cela soit accessible


In [91]:
import os

OUTPUT_DIR = "/content/drive/MyDrive/Projet_P4/Data_p4_clean"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df_transactions.to_csv(os.path.join(OUTPUT_DIR, "transactions_clean.csv"), index=False, encoding="utf-8")

df_customers.to_csv(os.path.join(OUTPUT_DIR, "customers_clean.csv"), index=False, encoding="utf-8")

df_products.to_csv(os.path.join(OUTPUT_DIR, "products_clean.csv"), index=False, encoding="utf-8")


